In [1]:
# !pip install requests
# !pip install bs4
# !pip install selenium

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1279 sha256=2c26dc02db380cf66445224e1216655902e9648c12657bf155b3442410e7923b
  Stored in directory: c:\users\yujung\appdata\local\pip\cache\wheels\0a\9e\ba\20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4


In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

In [3]:
# without window options
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)

In [5]:
driver = webdriver.Chrome('chromedriver')
driver.get("https://smartstore.naver.com/jainystore/products/2455969597")

In [6]:
driver.find_element_by_xpath("//div[@class='detail_tab_floatable']//li[@class='item']/a[@class='link N=a:tab.review']").click()

In [7]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
review = soup.find_all('p',class_='review_text _review_text')
review = [each_line.get_text().strip() for each_line in review[:20]]
print(review)

['BEST\n저는 여태까지 일반 베개만 사용하다가 불편함을 느껴서 이번에 나를 위해 괜찮은 베개 하나 사보자! 해서 경추베개를 찾아보다가 이 잠솔솔 베개를 보게 되었어요\n\n수많은 베개를 찾아봤는데 그 중에서도 잠솔솔이라는 베개를 구매한 이유는 \n\n✔️믿을 수 있는 많은 후기들\n✔️속커버는 생활 방수 기능과 땀,진드기 차단 \n✔️겉커버는 세탁가능하고, 별도로 구매도 가능하다는 점\n✔️100% 국내산 메모리폼, 라돈 및 유해물질 불검출\n✔️\n\n이라는 점이었어요!\n\n잠솔솔 메쉬 아이보리를 일요일 밤에 주문했구요 이틀 후인 화요일 낮에 베개를 받았어요! 손잡이가 있는 상자라 들기가 편하더라고요 선물용으로 좋겠다고 생각했어요 \n베개색상은 아이보리보단 흰색에 가까웠어요! 때가 좀 탈 거 같긴 하지만 겉커버를 별도로 세탁도 가능하니까 괜찮았어요!!\n\n잠솔솔베개를 사용한 첫날에는 우선 안정감있다는 느낌을 받았어요 그런데 적응이 안돼서 그런지 좀 딱딱하다고 느껴지더라구요 그래도 베개가 너무 푹신한 것보다 탄탄해야 목 부분을 더 잘 받쳐주고 좋을 것 같아요 다음날에는 비닐을 벗기고 사용했는데 제가 냄새에 예민해서 그런지 냄새가 계속 나서 잠자기 힘들더라고요 냄새나는건 말려주면 날아갈거 같아요 베개 자체는 굉장히 편했어요 :) 평소에 저는 정면으로, 양옆으로 돌아서 이리저리 자세를 바꿔가며 자는데 옆으로 자다보면 어깨랑 귀가 너무 눌려서 자고 일어나면 어깨가 좀 뻐근?해서 좀 많이 불편했어요 근데 잠솔솔 베개는 어깨랑 귀가 눌리지 않고 바른 자세로 편한게 자는 거 같아서 너무 좋아요\n\n\n이상 잠솔솔 베개를 사용해 본 후기를 정리하자면!! \n\n장점\n✔️옆으로 누웠을 때 귀와 어깨가 눌리지 않는 점\n✔️베개가 너무 높지도 낮지도 않은 적당한 높이\n✔️가로 사이즈가 넉넉하여 안정적임\n✔️바른 자세를 유지해줘서 숙면에 도움이 됨\n✔️커버 재질이 매쉬라서 통풍이 잘 되고 관리가 쉬움\n\n아쉬움 점\n✔️겉커버(밝은 색)에 때가 탈 우려\n✔

In [9]:
page = 3
path = f"//nav[@class='module_pagination _review_list_page']//a[@class='page number N=a:rvp.page _click(nmp.front.sellershop.product.review.list.goPage({page})) _stopDefault']"
driver.find_element_by_xpath(path).click()

In [10]:
driver.quit()